# Tarea 4
### Luis Armando Pérez: luisarpere@gmail.com, github:LuisArmandoPerez
### Luis Felipe Gomez López: L.Felipe_Gomez@ciencias.com.mx, github:LFelipeGomez
### Santiago Ontañón Sánchez: sontanon@gmail.com, github:sontanon

# Diferenciación automática (o algorítmica): Parte 1

### NOTA

Este notebook se iniciará en la clase y **debe** hacerse en equipo de dos personas (máximo tres). La resolución completa de los ejercicios debe ser enviada como "Tarea4.ipynb".

## Motivación

En el [notebook anterior](https://github.com/lbenet/2016-2_TSFisicaComputacional/blob/master/notas_clase/06_Derivacion_numerica.ipynb), vimos que hay diferentes formas de implementar la derivación de una función $f(x)$ en un punto $x_0$ y que éstas dependen de un parámetro $h$, que es la separación entre puntos cercanos. 

Vimos que el error absoluto en términos de $h$, cuando $h\to 0$, tiene un comportamiento distinto, y que de hecho puede ser "contaminado" por errores de numéricos.

Se obtuvo que:

- La "derivada derecha" converge *linealmente* al $h\to 0$ al valor correcto de la derivada, es decir, el error es proporcional a $\mathcal{O}(h)$. Sin embargo, para valores *suficientemente* pequeños de $h$, el valor obtenido de la derivada deja de tener sentido ya que pierde exactitud.

- La "derivada simétrica" converge *cuadráticamente*, es decir, el error escala como $\mathcal{O}(h^2)$. Al igual que la derivada derecha, para $h$ suficientemente pequeña errores "de cancelación" debidos a la diferencia en la definición hacen que el resultado pierda sentido.

- La definición de la "derivada compleja" también converge *cuadráticamente*; a diferencia de las dos definiciones anteriores *no* exhibe problemas al considerar valores de $h$ muy pequeños.

Los puntos anteriores muestran que, la manera de implementar un algoritmo (concreto) cualquiera en la computadora es importante respecto a cuestiones de la convergencia y del manejo de errores numéricos. En este sentido, la "derivada compleja" da el resultado que más se acerca al exacto, incluso para valores de $h \sim 10.^{-16}$.

La pregunta es si podemos ir más allá y obtener el valor exacto, esto es, el que más se acerca al valor obtenido con números reales excepto por cuestiones de redondeo.

## Diferenciación automática (o algorítmica)

Citando a [wikipedia](https://en.wikipedia.org/wiki/Automatic_differentiation): ``Automatic differentiation (AD), also called algorithmic differentiation or computational differentiation, is a set of techniques to numerically evaluate the derivative of a function specified by a computer program.''

Diferenciación automática **no es** diferenciación numérica. Está basada en cálculos numéricos (evaluación de funciones por la computadora), pero **no** usa ninguna de las definiciones por diferencias finitas, como las que vimos la clase anterior.

Para ilustrar su funcionamiento, empezaremos recordando cómo funcionan los números complejos: $z = a + \mathrm{i} b$.

Uno puede definir todas las operaciones aritméticas de la manera natural (a partir de los números reales), manteniendo las expresiones con $\mathrm{i}$ factorizada; en el caso de la multiplicación y división, simplemente recordamos que $\mathrm{i}^2=-1$.

Por ejemplo, tenemos que, para $z = a + \mathrm{i} b$ y $w = c + \mathrm{i} d$
\begin{eqnarray*}
z + w & = & (a + \mathrm{i} b) + (c + \mathrm{i} d) = a + c + \mathrm{i}(b + d),\\
z \cdot w & = & (a + \mathrm{i} b)\cdot (c + \mathrm{i} d) \\
  & = & ac + \mathrm{i} (ad+bc) + \mathrm{i}^2 b d
 = ac - b d + \mathrm{i} (ad+bc).
\end{eqnarray*}

Por último, vale también la pena recordar que, $\mathbb{C}$ es *isomorfo* a $\mathbb{R}^2$, esto es, hay un mapeo biyectivo de $ z \leftrightarrow (a, b)$.

Volviendo a la cuestión de la diferenciación automática, la idea es introducir un par ordenado donde la primer
componente es el valor de una función $f(x)$ evaluada en $x_0$, y la segunda es el valor de su derivada evaluada
en el mismo punto. Esto es, definimos a los *duales* como:

\begin{equation}
\vec{f}(x_0) = \big( f_0, f'_0\big) = f_0 + \hat{\mathbf{x}}\, f'_0,
\end{equation}

donde $f_0 = f(x_0)$ y $f'_0=\frac{d f}{d x}(x_0)$ y, en la segunda igualdad, $\hat{\mathbf{x}}$ sirve para indicar la segunda componente del par ordenado. (En un sentido que se precisará más adelante, $\hat{\mathbf{x}}$ se comporta de una manera semejante a $\mathrm{i}$ para los números complejos, distinguiéndose en $\hat{\mathbf{x}}^2$.)

En particular, para la función constante $f(x)=c$ se cumple que el dual asociado es $\vec{c}(x_0)=(c,0)$, y para la función identidad $f(x)=x$ obtenemos $\vec{x}(x_0)=(x_0,1)$. Vale la pena aquí enfatizar que la función identidad es precisamente la variable independiente, respecto a la que se está derivando.

### Ejercicio

Implementen una nueva estructura paramétrica (`type`) que defina los duales, donde el parámetro debe ser un subtipo de `Real` (ver la siguiente celda). La parte que identifica a $f_0$ será llamada `fun`, y la correspondiente a $f'_0$ será `der`.

La definición debe incluir métodos que sean compatibles con las dos propiedades arriba mencionadas, es decir, que el dual de una constante (cualquier número real) sea $(c,0)$, y que el de la variable independiente sea $(x_0,1)$. Para lo segundo definiremos una función `xdual` con la propiedad mencionada.

In [1]:
workspace()

In [2]:
"""
Dual

Es un tipo parametrizado por subtipos Reales. Posee entradas fun y der.
"""
type Dual{T<:Real}
    # Se declara el parámetro y las entradas fun y der.
    fun::T
    der::T
end

Como se puede ver, no podemos meter cualquier cosa en nuestra `Dual`:

In [3]:
typeof(Dual("Santiago",1.0...))

LoadError: LoadError: MethodError: `convert` has no method matching convert(::Type{Dual{T<:Real}}, ::ASCIIString, ::Float64)
This may have arisen from a call to the constructor Dual{T<:Real}(...),
since type constructors fall back to convert methods.
Closest candidates are:
  Dual{T<:Real}(!Matched::T<:Real, ::T<:Real)
  call{T}(::Type{T}, ::Any)
  convert{T}(::Type{T}, !Matched::T)
while loading In[3], in expression starting on line 1

In [4]:
# Con este método se promueven fun y der para ser del mismo tipo usando
# promote y luego usando un splat dentro del tipo Dual
Dual(fun,der) = Dual(promote(fun, der)...)

Dual{T<:Real}

Por ejemplo, si metemos pi que es de tipo irracional y 1 de tipo entero:

In [5]:
Dual(pi, 1)

Dual{Float64}(3.141592653589793,1.0)

Se ve que el Dual ya tiene los mismos subtipos `Float64`.

In [6]:
# Aqui se define un método que garantiza que el dual de un número cumple lo requerido
Dual(a::Real) = Dual(a,0...)

Dual{T<:Real}

Por ejemplo:

In [7]:
Dual(pi)

Dual{Float64}(3.141592653589793,0.0)

In [8]:
# Aqui se define la función `xdual`, que se usará para identificar la variable independiente
function xdual(x)
    return Dual(x, 1...) # Como consiste en la variable independiente, es la identidad.
end

xdual (generic function with 1 method)

Una prueba sencilla:

In [9]:
xdual(2.0)

Dual{Float64}(2.0,1.0)

In [10]:
# Muestren que su código funciona con tests adecuados; para los detalles ver 
# http://julia.readthedocs.org/en/release-0.4/stdlib/test/
using Base.Test
x2 = xdual(2.0)
# Aqui vienen varios tests de cada una de las funciones que implementaron. 
# Para ver más pruebas consulte el archivo anexo runtest.jl
@test x2.fun == 2.0
@test x2.der == 1.0

x3 = xdual(2.0+pi)
@test x3.fun == (2.0 + pi)
@test x3.der == 1.0

x4 = Dual(7.9)
@test x4.fun == 7.9
@test x4.der == 0.0

## Aritmética de duales

De la definición, y recordando el significado de cada una de las componentes del par doble, tenemos que las operaciones aritméticas quedan definidas por:

\begin{eqnarray}
    \vec{u} \pm \vec{w} &=& \big( u_0 \pm w_0,\, u'_0\pm w'_0 \big),\\
    \vec{u} \times \vec{w} &=& \big( u_0 \cdot w_0,\, u_0 w'_0 +  w_0 u'_0 \big),\\
    \frac{\vec{u}}{\vec{w}} &=& \big( \frac{u_0}{w_0},\, \frac{ u'_0 - (u_0/w_0)w'_0}{w_0}\big),\\
    {\vec{u}}^\alpha &=& \big( u_0^\alpha,\, \alpha u_0^{\alpha-1} u'_0 \big).\\
\end{eqnarray}    

Además, están los operadores unitarios, que satisfacen:
\begin{equation}
    \pm \vec{u} = \big(\pm u_0, \pm u'_0 \big).
\end{equation}    


### Ejercicio

Implementen *todas* las operaciones aritméticas. Incluyan tests que muestren que cada una de ellas está bien definida, y que sus resultados dan valores consistentes.

In [20]:
import Base: +, -, *, /, ^, ==

# Aqui se implementan los métodos necesarios para cada función
# Las operaciones de suma y resta incluyen el caso de sumar/restar duales 
# a números reales.
+(a::Dual, b::Dual) = Dual((a.fun + b.fun),(a.der + b.der)...)
+(a::Real, b::Dual) = Dual(a) + b
+(a::Dual, b::Real) = b + a
-(a::Dual, b::Dual) = Dual((a.fun - b.fun),(a.der - b.der)...)
-(a::Real, b::Dual) = Dual(a)-b
-(a::Dual, b::Real) = a - Dual(b)
# Del mismo modo, la operación de multiplicación incluye multiplicar por un
# número real, convirtiéndolo a su dual y luego multiplicando por el método
# definido de multiplicación entre duales.
*(a::Dual, b::Dual) = Dual((a.fun * b.fun),(a.fun * b.der + b.fun * a.der)...)
*(a::Real, b::Dual) = Dual(a) *b
*(a::Dual, b::Real) = b*a
/(a::Dual, b::Dual) = Dual((a.fun / b.fun),((a.der-(a.fun / b.fun)*b.der)/b.fun)...)
# Para la exponenciación, Julia nos pide que definamos primero la exponenciación
# con un entero y luego con un número real.
^(a::Dual,ex::Integer) = Dual((a.fun^ex), (ex*a.fun^(ex-1)*a.der)...)
^(a::Dual,ex::Real) = Dual((a.fun^ex), (ex*a.fun^(ex-1)*a.der)...)
# Las operaciones unitarias son sencillas de ver.
+(a::Dual) = a
-(a::Dual) = Dual(-a.fun, -a.der...)
# Finalmente se agrega un método para comparar dos duales que nos servirá en los tests.
==(a::Dual, b::Dual) = (a.fun == b.fun && a.der == b.der) ? true : false

== (generic function with 110 methods)

In [21]:
# Aqui se incluyen algunas pruebas, nuevamente, hay más preubas comentadas
# en el archivo runtests.jl
@test -xdual(2.0) == Dual(-2.0,-1.0...)
@test xdual(2.0) + Dual(pi) == Dual(2.0 + pi, 1.0)
@test xdual(2.0)^1.0 == xdual(2.0)

Ahora probaremos la infraestructura que han desarrollado hasta ahora. En particular, usaremos la misma función que se
utilizón en la clase anterior:

\begin{equation}
    f_\textrm{test}(x) = 3x^3 - 2,
\end{equation}

y la idea es calcular la derivada en $x_0=1$.

In [22]:
f_test(x) = 3*x^3 - 2

f_test (generic function with 1 method)

In [24]:
# Nuestra variable independiente está evaluada en 1.0
xdual(1.0)

Dual{Float64}(1.0,1.0)

In [16]:
# Del resultado anterior, obtengan el valor de la derivada

In [27]:
# Es claro que simplemente tenemos que evaluar f_test(xdual(1.)) y la
# derivada será la segunda parte de la dual.
f_test(xdual(1.0))

Dual{Float64}(1.0,9.0)

In [29]:
f_test(xdual(1.0)).der

9.0

La derivada es claramente 9, como se puede verificar de manera analítica

$$f'_{\text{test}}(x) = 9x^2$$

Consideremos otra función racional:

$$
g_\textrm{test}(x) = (3x^2-8x+5)/(7x^3-1)
$$

cuya derivada queremos calcular en $x_0=1$. Según [WolframAlpha](http://www.wolframalpha.com/input/?i=D%5B+%283x%5E2-8x%2B5%29%2F%287x%5E3-1%29+%2C+x+%5D+%2F.+x-%3E+1), el resultado exacto es $-1/3$.


In [30]:
gtest(x) = (3x^2-8x+5)/(7x^3-1)

gtest (generic function with 1 method)

In [31]:
gtest( xdual(1) )

Dual{Float64}(0.0,-0.3333333333333333)

El resultado anterior es la representación en números de punto flotante de $-1/3$. La pregunta es si podemos obtener
el resultado exacto.

In [34]:
# Evalúen `gtest` de manera que el resultado sea el exacto
# Si hacemos las operaciones con números reacionales, el resultado debe ser
# racional porque la función gtest está definida con enteros y la promoción
# debe aventar el subtipo mayor que son los números racionales. De esta manera:
gtest(xdual(1//1)).der

-1//3

Para entender cómo funciona esto, evaluaremos explícitamente y paso a paso $g_\textrm{test}(x)$ en $\vec{x_0} = (1,1)$:

\begin{equation}
\vec{g_\textrm{test}}(\vec{x}) = \frac{\vec{3}\cdot{\vec{x}}^2-\vec{8}\cdot\vec{x}+\vec{5}}{\vec{7}\cdot{\vec{x}}^3-\vec{1}}
\end{equation}


\begin{eqnarray}
\vec{g_\textrm{test}}(\vec{x}) & = & 
\frac{(3,0)\cdot{(1,1)}^2-(8,0)\cdot(1,1)+(5,0)}{(7,0)\cdot{(1,1)}^3-(1,0)}\\
& = & \frac{(3,0)\cdot(1,2)-(8,0)\cdot(1,1)+(5,0)}{(7,0)\cdot(1,3)-(1,0)}
\end{eqnarray}


\begin{equation}
\vec{g_\textrm{test}}(\vec{x}) = 
\frac{ (3,6)-(8,1)+(5,0)}{(7,21)-(1,0)} = \frac{ \Big(3-8+5, 6-8\Big)}{\Big( 7-1,21\Big)} = 
\frac{ \Big(0, -2\Big)}{\Big( 6,21\Big)} 
\end{equation}

\begin{equation}
\vec{g_\textrm{test}}(\vec{x}) = \Big(\frac{0}{6}, \frac{(-2)-21\cdot(0/6)}{6} \Big) = 
\Big(0, -\frac{1}{3}\Big). \Box
\end{equation}


## Modulos y "runtests.jl" en julia

### Ejercicio

Para reutilizar el código que han hecho en este notebook, y de hecho seguirlo desarrollando, conviene ponerlo dentro de un módulo. Para hacer esto, deberán copiar todo el código necesario (y que aparece en la resolución de los ejercicios anteriores) en un archivo coyo nombre será "AutomDiff.jl" y cuya estructura es:

```julia
# Aqui viene una explicación de lo que se hace en el módulo, los autores y la fecha

# La siguiente instrucción sirve para *precompilar* el módulo
__precompile__(true)

module AD
    import Base: +, -, *, /, ^
    
    export Dual, xdual
    
    # Aqui viene TODO el código que implementaron

end
```

Todas las pruebas deberán ser incluídas en un archivo llamado "runtest.jl", y su estructura es:

```julia
# Este archivo incluye los tests del módulo AD

using Base.test
using AD

# A continuación vienen los tests que implementaron y que deben ser suficientemente exhaustivos


```

Estos dos archivos deben incluirlos en el envío de su tarea.

**Nota** Los archivos se importan usando:

```
    julia> include("AutomDiff.jl")
    AD
    
    julia> include("runtest.jl")
```

Como se puede ver, después de importar AutomDiff.jl, Julia nos dice que está disponible el módulo AD. Para el archivo de tests, el hecho de que la salida sale en blanco nos dice que todos los tests corrieron de manera exitosa. Después, ya teniendo el módulo AD podemos correr:
```
    julia> f(x) = x^2 - 2x + 1; f(xdual(1.0)).der
    0.0
    
    julia> g(x) = (3x^2 - 4x + 1)/(6x^2 - 9x - 8); g(xdual(1//1)).der
    -2//11
```

Para demostrarlo vamos a meterlo  este notebook y vamos a limpiar el `workspace`:

In [6]:
workspace()

In [7]:
include("AutomDiff.jl")

AD

In [8]:
include("runtest.jl")

In [9]:
f(x) = x^2 - 2x + 1; f(xdual(1.0)).der

0.0

In [10]:
g(x) = (3x^2 - 4x + 1)/(6x^2 - 9x - 8); g(xdual(1//1)).der

-2//11